# Numerical Benchmarks

In [1]:
import sys
sys.path.insert(0, '../../src_tf/')

import numpy as np
import qiskit as qk
import matplotlib.pyplot as plt
import multiprocessing as mp
import random
import pickle

from qiskit.quantum_info import DensityMatrix, random_unitary
from qiskit.quantum_info import Operator
from scipy.linalg import sqrtm
from tqdm.notebook import tqdm

from loss_functions import *
from optimization import *
from quantum_channel import *
from kraus_channels import *
from lindblad_channels import *
from quantum_tools import *
from experimental import *
from spam import *

#np.set_printoptions(threshold=sys.maxsize)
np.set_printoptions(precision=4)

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

## Recover SPAM and Map

### Generate True Model, Full POVM and Inital

In [5]:
def generate_spam_benchmark(n=3, c1=1, c2=1):
    d = 2**n

    init_target = InitialState(d, c=c1)
    povm_target = POVM(d, c=c2)

    spam_target = SPAM(init = init_target,
                       povm = povm_target)
    
    return spam_target


def generate_spam_data(spam_target, N_spam=None, shots=1024):
    n = int(np.log2(spam_target.d))
    inputs_spam, _ = generate_pauliInput_circuits(n)
    N_spam = inputs_spam.shape[0]

    state = tf.repeat(spam_target.init.init[None,:,:], N_spam, axis=0)
    targets_spam = measurement(state, U_basis = inputs_spam, povm = spam_target.povm.povm)

    #add noise
    targets_spam = add_shot_noise(targets_spam, shots = shots)
    return inputs_spam, targets_spam


def generate_map_data(channel_target, spam_target, N_map=None, shots=1024):
    n = int(np.log2(channel_target.d))
    inputs_map, _ = generate_pauli_circuits(n = n, 
                                            circuit_target=None,  
                                            trace=False,
                                            N=N_map)
    U_prep, U_basis = inputs_map

    N_map = U_prep.shape[0]
    state = tf.repeat(tf.expand_dims(spam_target.init.init, axis=0), N_map, axis=0)
    state = apply_unitary(state, U_prep)
    state = channel_target.apply_channel(state)
    targets_map = measurement(state, U_basis, spam_target.povm.povm)
    
    #add noise
    targets_map = add_shot_noise(targets_map, shots = shots)

    return inputs_map, targets_map


def model_pipeline(channel_target = None, shots=1024, N_map = None, num_iter_spam=None, num_iter_map=None):
    # Make Benchmark
    #################################################################################
    n = 3
    d = 2**n
    
    spam_target = generate_spam_benchmark(n=n, c1=0.9, c2=0.9)
    inputs_spam, targets_spam = generate_spam_data(spam_target, N_spam=None, shots=shots)

    inputs_map, targets_map = generate_map_data(channel_target, spam_target, N_map=N_map, shots=shots)
    #################################################################################

    # Fit Models
    #################################################################################
    spam_model = SPAM(init = InitialState(d, c=0.9),
                      povm = CorruptionMatrix(d, c=0.9),
                      optimizer = tf.optimizers.Adam(learning_rate=0.01))
    
    #spam_model.pretrain(300, verbose=False)

    spam_model.train(inputs = inputs_spam,
                     targets = targets_spam,
                     num_iter = num_iter_map,
                     verbose = False,
                    )

    model = ModelQuantumMap(channel = KrausMap(d = d, 
                                               rank = d**2,
                                               spam = spam_model,
                                              ),
                            loss_function = ProbabilityMSE(),
                            optimizer = tf.optimizers.Adam(learning_rate=0.01),
                            logger = Logger(loss_function = ProbabilityMSE(),
                                            loss_function_val = channel_fidelity_loss),
                           )

    model.train(inputs = inputs_map,
                targets = targets_map,
                inputs_val = None,
                targets_val = [channel_target],
                num_iter = num_iter_map,
                N = 500,
                )
    #################################################################################
    model.optimizer = None
    spam_model.optimizer = None
    model.inputs = None
    model.targets = None
    model.inputs_val = None
    model.targets_val = None

    return model

## Fit Models

In [6]:
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

n = 3
d = 2**n

channel_target1 = CompactLindbladMap(d, 1, 1, 0.1)
model1 = model_pipeline(channel_target1, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)

channel_target2 = CompactLindbladMap(d, 16, 10000, 0.001)
model2 = model_pipeline(channel_target2, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)

channel_target3 = CompactLindbladMap(d, 16, 100, 0.001)
model3 = model_pipeline(channel_target3, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)

channel_target4 = CompactLindbladMap(d, 8, 1, 0.01)
model4 = model_pipeline(channel_target4, 
                        shots=1024, 
                        N_map=2000-6**n, 
                        num_iter_spam = 4000,
                        num_iter_map = 4000)


saver([[channel_target1, channel_target2, channel_target3, channel_target4],
       [model1, model2, model3, model4]], data_path("atypical_maps_benchmark.model"))

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0007307941044150015


  0%|          | 0/4000 [00:00<?, ?it/s]

0.25219703942326327 -0.09635478056450521
0.05936668417387033 -0.3897203431018232
0.01626204628092945 -0.6900737721001878
0.009364847133204767 -0.8125720706948297
0.007762427042397627 -0.8649727278402102
0.007186614383928052 -0.8923329219240974
0.006916551012235016 -0.9083439039622644
0.006768251138233771 -0.9184019421716787
0.006670954742639465 -0.925020428973419
0.006614791988384132 -0.9296590657339426
0.006575300614113096 -0.9330444340848834
0.006550429250284933 -0.9352248155157346
0.006534157267362303 -0.9369588420269934
0.006514320739561761 -0.9381903820474227
0.006505178815576172 -0.9390415183861615
0.006500234404517798 -0.9395386050131186
0.0064958516832092335 -0.940393606016206
0.00649194056023336 -0.940534912332515
0.006489481632929697 -0.9409592613218083
0.006488976383631382 -0.9414234510261699
0.006493595928101083 -0.9408849151627675
0.006482729856534139 -0.9415754738377864
0.006483107314069715 -0.9411819064448612
0.0064936184256857675 -0.9418919347192211
0.006481942030748786

  0%|          | 0/4000 [00:00<?, ?it/s]

...

0.5211334257396193 -0.15669000604136157
0.14820858571366072 -0.3991778142124804
0.01743758869760857 -0.7120801764186422
0.00923478075974717 -0.7961654859967686
0.007708982236211278 -0.8307582162033089
0.006965808764758562 -0.8532407831780089
0.006566628945879687 -0.8686264644078112
0.006329938619140391 -0.8795806733959667
0.006175009925881089 -0.8874307475839033
0.006077292916300086 -0.8930811390884207
0.006002870625820707 -0.8969406785193907
0.005950565531453715 -0.9001601773335587
0.005905904479148971 -0.9021531567185517
0.0058734984893033205 -0.9031566813540096
0.0058526339094381674 -0.9044639788733164
0.005844644085251112 -0.9051338558154223
0.005821365659801832 -0.9054269896678332
0.005808958525385819 -0.9058529754515146
0.005799928905974782 -0.9054526450828853
0.005790281946304662 -0.9053525643698274
0.005793731573540951 -0.9054678302334729
0.005787215528522851 -0.9053134438450214
0.0057788253151059826 -0.9043152516239688
0.005771753625348487 -0.9043603598577129
0.005780798388620

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0007115445227517261


  0%|          | 0/4000 [00:00<?, ?it/s]

0.11842445571074112 -0.4363524927727841
0.019603552560397877 -0.6557301187355249
0.008418981551772875 -0.7959621458019565
0.006514829432458024 -0.8476202567906478
0.00594618851655267 -0.8696724454991128
0.0056847798184669115 -0.8797123258911127
0.005531743546723922 -0.8838946867302622
0.005443877515683808 -0.8844743895596873
0.005385399677277801 -0.8838313549051592
0.005347821377560531 -0.8817032993531831
0.005316718600305249 -0.8798896938176238
0.0052981805692433985 -0.8774436880084125
0.005280200312809362 -0.8755016016610699
0.00527021424146344 -0.8728890173752495
0.005258510075964948 -0.8707613037898464
0.005253423275899945 -0.8682689621976099
0.005243908976054245 -0.8664470031062916
0.005239763293637459 -0.8647144970159042
0.005243451167591512 -0.8626666367816326
0.0052383391966884674 -0.861033054030168
0.00524160728118691 -0.8594081559173108
0.00524205533119871 -0.8581092214365169
0.005238078145906508 -0.8565194632860709
0.005241180937250075 -0.855279748494934
0.005243096339077500

## Sample and Noise Sensitivity

In [7]:
n = 3
d = 2**n
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

channel_target = CompactLindbladMap(d, 1, 1, 0.1)
model_ns1 = model_pipeline(channel_target, 
                           shots=256, 
                           N_map=256, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000)


model_ns2 = model_pipeline(channel_target, 
                           shots=1024, 
                           N_map=1024, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000)

model_ns3 = model_pipeline(channel_target, 
                           shots=4096, 
                           N_map=4096, 
                           num_iter_spam = 4000,
                           num_iter_map = 4000)

saver([[channel_target],
       [model_ns1, model_ns2, model_ns3]], data_path("atypical_maps_robustness.model"))

  0%|          | 0/4000 [00:00<?, ?it/s]

0.0029888454190854148


  0%|          | 0/4000 [00:00<?, ?it/s]

0.27177982547349966 -0.09635478056450521
0.059989024633783514 -0.3411237006232957
0.02706846113237093 -0.5700153110733187
0.02007195284160605 -0.6786044992078635
0.017922502613138755 -0.7279848402352672
0.017087591661281244 -0.7519146561862163
0.016712243785150953 -0.7641759089571696
0.016524976046139905 -0.7707537994888708
0.016424322513810593 -0.7744117093507505
0.016367267319698148 -0.7764985131236316
0.016333624717794024 -0.7777063156161176
0.016313157779765164 -0.7784065165666025
0.01630036973440923 -0.7788063927634858
0.01629218261610941 -0.7790253332384134
0.016286818082133094 -0.7791337910945986
0.016283223476998742 -0.7791741582635295
0.01628076300855105 -0.7791723996473592
0.016279045409625763 -0.779144714687066
0.016277825214391528 -0.779101405971468
0.016276945184101124 -0.7790491572339634
0.01627630231254386 -0.7789923645786324
0.01627582755101201 -0.7789339570262864
0.016275473583567034 -0.7788758673599241
0.01627520734989493 -0.7788193625597344
0.016275005382160274 -0.77

  0%|          | 0/4000 [00:00<?, ?it/s]

...

0.26416944296960354 -0.09538214878993144
0.05888477792327918 -0.3860753476942293
0.01571131272108863 -0.6772817434413269
0.008895787593283067 -0.8007864953344385
0.0072632923101247065 -0.8547745046304207
0.006661962375683734 -0.8829172209741485
0.0063766145207387465 -0.899351439946464
0.006219784820202752 -0.909816760649954
0.006123575362174361 -0.9166695755822153
0.006064333556936526 -0.92133817753901
0.006022442441726448 -0.9246634366682576
0.0059958349362247 -0.927012793669272
0.00597369158062209 -0.9286796460333041
0.005969860488885249 -0.9298234027224311
0.005956841941706538 -0.9306993145688508
0.005943627230147731 -0.9314977919253765
0.005938961363804056 -0.9319336792631445
0.005932546005671007 -0.9325534796355631
0.00593701702846372 -0.9327207797877619
0.005939095599635969 -0.9329465095065084
0.005929779552912496 -0.9335091268958849
0.005932136667840121 -0.9332716302847383
0.005934255791286029 -0.933422475450592
0.005937443027791837 -0.9336852482797119
0.0059319687419853205 -0.9

  0%|          | 0/4000 [00:00<?, ?it/s]

0.00023675669485870437


  0%|          | 0/4000 [00:00<?, ?it/s]

0.2544929954433411 -0.09384857320242698
0.05818475103912477 -0.381054767444444
0.011721284967004371 -0.6881590825650501
0.004509101390412119 -0.8159779081612968
0.0029030013856560367 -0.871547028298427
0.0023650715079162553 -0.9012139206082678
0.002123448995225731 -0.9196674072072777
0.0019950343008764875 -0.9319651112470689
0.0019226603006550777 -0.9407796213300649
0.0018749592508533644 -0.9474307783086767
0.0018440735832067488 -0.9524339748657686
0.0018204791592853964 -0.9562985948931292
0.0018037094859862934 -0.9593305158569388
0.0017928164705002143 -0.9617710987812096
0.0017846178151814836 -0.9636140847748019
0.001778749601506604 -0.965155467776948
0.0017709808822758775 -0.9665185340427557
0.0017685713152739437 -0.9676064482784629
0.0017652024533916719 -0.9684160305464098
0.0017649105676294436 -0.9690256249701132
0.0017631839476927084 -0.9696101611441599
0.001762496570432329 -0.9701606079026486
0.0017612891958513938 -0.9704875485039567
0.0017585061003847352 -0.9708691295587752
0.00